# Test using Pyro

In [34]:
import torch

import pyro
import pyro.distributions as dist
import pyro.poutine as poutine
from pyro.infer import MCMC, NUTS

In [35]:
np.e ** 0.001

1.0010005001667084

In [60]:
    k1 = pyro.sample('k1', dist.Normal(np.log10(0.0016), 0.001))
    k_1 = pyro.sample('k_1', dist.Normal(np.log10(0.004), 0.001))
    kt = pyro.sample('kt', dist.Normal(np.log10(0.0012), 0.001))
    ke = pyro.sample('ke', dist.Normal(np.log10(0.0033), 0.001))
    Vr = pyro.sample('Vr', dist.Normal(np.log10(0.0012), 0.001))
    kx = pyro.sample('kx', dist.Normal(np.log10(1), 0.001))
    kh = pyro.sample('kh', dist.Normal(np.log10(0.0004), 0.001))
    k2 = pyro.sample('k2', dist.Normal(np.log10(0.082), 0.001))
    k_2 = pyro.sample('k_2', dist.Uniform(-4, 2))
    k3 = pyro.sample('k3', dist.Normal(np.log10(1.2), 0.001))
    k_3 = pyro.sample('k_3', dist.Uniform(-4, 2))
    res = getResidual(model, np.array(10 ** np.array([k1.item(), k_1.item(), kt.item(), ke.item(), Vr.item(), 
                                       kx.item(), kh.item(), k2.item(), k_2.item(), k3.item(), k_3.item()])))
    pyro.sample("obs", dist.Normal(res, 0.0001))

tensor(9.9749)

In [90]:
    k1 = pyro.sample('k1', dist.Normal(np.log10(0.0016), 0.001))
    k_1 = pyro.sample('k_1', dist.Normal(np.log10(0.004), 0.001))
    kt = pyro.sample('kt', dist.Normal(np.log10(0.0012), 0.001))
    ke = pyro.sample('ke', dist.Normal(np.log10(0.0033), 0.001))
    Vr = pyro.sample('Vr', dist.Normal(np.log10(0.0012), 0.001))
    kx = pyro.sample('kx', dist.Normal(np.log10(1), 0.001))
    kh = pyro.sample('kh', dist.Normal(np.log10(0.0004), 0.001))
    k2 = pyro.sample('k2', dist.Normal(np.log10(0.082), 0.001))
    k_2 = pyro.sample('k_2', dist.Uniform(-4, 2))
    k3 = pyro.sample('k3', dist.Normal(np.log10(1.2), 0.001))
    k_3 = pyro.sample('k_3', dist.Uniform(-4, 2))

    res = getResidual(model, 10 ** np.array([k1.item(), k_1.item(), kt.item(), ke.item(), Vr.item(), 
                                       kx.item(), kh.item(), k2.item(), k_2.item(), k3.item(), k_3.item()]))
    torch.ones(1) * res

tensor([9.9749])

In [91]:
def resModel(model):
    k1 = pyro.sample('k1', dist.Normal(np.log10(0.0016), 0.001))
    k_1 = pyro.sample('k_1', dist.Normal(np.log10(0.004), 0.001))
    kt = pyro.sample('kt', dist.Normal(np.log10(0.0012), 0.001))
    ke = pyro.sample('ke', dist.Normal(np.log10(0.0033), 0.001))
    Vr = pyro.sample('Vr', dist.Normal(np.log10(0.0012), 0.001))
    kx = pyro.sample('kx', dist.Normal(np.log10(1), 0.001))
    kh = pyro.sample('kh', dist.Normal(np.log10(0.0004), 0.001))
    k2 = pyro.sample('k2', dist.Normal(np.log10(0.082), 0.001))
    k_2 = pyro.sample('k_2', dist.Uniform(-4, 2))
    k3 = pyro.sample('k3', dist.Normal(np.log10(1.2), 0.001))
    k_3 = pyro.sample('k_3', dist.Uniform(-4, 2))

    res = getResidual(model, 10 ** np.array([k1.item(), k_1.item(), kt.item(), ke.item(), Vr.item(), 
                                       kx.item(), kh.item(), k2.item(), k_2.item(), k3.item(), k_3.item()]))

    return torch.ones(1) * res

def conditioned_model(resModel, model, y):
    return poutine.condition(resModel, data={"obs": y})(model)


In [ ]:
nuts_kernel = NUTS(conditioned_model, jit_compile=False)
mcmc = MCMC(nuts_kernel, num_samples=1000, warmup_steps=1000, num_chains=4)
mcmc.run(resModel, model, torch.zeros(1))
mcmc.summary(prob=0.68)

# Learn Pyro

In [1]:
import torch
import pyro

pyro.set_rng_seed(101)

In [2]:
loc = 0.   # mean zero
scale = 1. # unit variance
normal = torch.distributions.Normal(loc, scale) # create a normal distribution object
x = normal.rsample() # draw a sample from N(0,1)
print("sample", x)
print("log prob", normal.log_prob(x)) # score the sample from N(0,1)

sample tensor(-1.3905)
log prob tensor(-1.8857)


In [3]:
def weather():
    cloudy = torch.distributions.Bernoulli(0.3).sample()
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = torch.distributions.Normal(mean_temp, scale_temp).rsample()
    return cloudy, temp.item()

In [4]:
x = pyro.sample("my_sample", pyro.distributions.Normal(loc, scale))
print(x)

tensor(-0.8152)


In [5]:
def weather():
    cloudy = pyro.sample('cloudy', pyro.distributions.Bernoulli(0.3))
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = pyro.sample('temp', pyro.distributions.Normal(mean_temp, scale_temp))
    return cloudy, temp.item()

In [6]:
def weather():
    cloudy = pyro.sample('cloudy', pyro.distributions.Bernoulli(0.3))
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]
    temp = pyro.sample('temp', pyro.distributions.Normal(mean_temp, scale_temp))
    return cloudy, temp.item()

for _ in range(3):
    print(weather())

('cloudy', 64.5440444946289)
('sunny', 94.37557983398438)
('sunny', 72.5186767578125)


In [7]:
def ice_cream_sales():
    cloudy, temp = weather()
    expected_sales = 200. if cloudy == 'sunny' and temp > 80.0 else 50.
    ice_cream = pyro.sample('ice_cream', pyro.distributions.Normal(expected_sales, 10.0))
    return ice_cream

In [8]:
def geometric(p, t=None):
    if t is None:
        t = 0
    x = pyro.sample("x_{}".format(t), pyro.distributions.Bernoulli(p))
    if x.item() == 1:
        return 0
    else:
        return 1 + geometric(p, t + 1)

print(geometric(0.5))

0


In [9]:
def normal_product(loc, scale):
    z1 = pyro.sample("z1", pyro.distributions.Normal(loc, scale))
    z2 = pyro.sample("z2", pyro.distributions.Normal(loc, scale))
    y = z1 * z2
    return y

def make_normal_normal():
    mu_latent = pyro.sample("mu_latent", pyro.distributions.Normal(0, 1))
    fn = lambda scale: normal_product(mu_latent, scale)
    return fn

print(make_normal_normal()(1.))

tensor(2.1493)


In [11]:
import argparse
import logging

import torch

import pyro
import pyro.distributions as dist
import pyro.poutine as poutine
from pyro.infer import MCMC, NUTS

In [ ]:
def resModel():
    

In [26]:
pyro.sample('eta', dist.Normal(torch.zeros(10), torch.ones(10)))

tensor([ 1.5410, -0.2934, -2.1788,  0.5684, -1.0845, -1.3986,  0.4033,  0.8380,
        -0.7193, -0.4033])

In [27]:
pyro.sample('mu', dist.Normal(torch.zeros(1), 10 * torch.ones(1)))

tensor([-5.9664])

In [28]:
pyro.sample('tau', dist.HalfCauchy(scale=25 * torch.ones(1)))

tensor([12.2214])

In [31]:
eta = pyro.sample('eta', dist.Normal(torch.zeros(10), torch.ones(10)))
mu = pyro.sample('mu', dist.Normal(torch.zeros(1), 10 * torch.ones(1)))
tau = pyro.sample('tau', dist.HalfCauchy(scale=25 * torch.ones(1)))

theta = mu + tau * eta

In [32]:
theta

tensor([-21.0833, -14.9126, -29.6163, -26.7419, -22.3080, -26.3238, -19.2022,
        -29.7435, -21.0826, -19.0633])

In [34]:
pyro.sample("obs", dist.Normal(theta, 1))

tensor([-20.8243, -16.6829, -29.4021, -27.2801, -21.7200, -24.7179, -18.7743,
        -30.4211, -20.0404, -21.0145])

In [49]:
    eta = pyro.sample('eta', dist.Normal(torch.zeros(10), torch.ones(10)))
    mu = pyro.sample('mu', dist.Normal(torch.zeros(1), 10 * torch.ones(1)))
    tau = pyro.sample('tau', dist.HalfCauchy(scale=25 * torch.ones(1)))

    theta = mu + tau * eta
    theta
    #pyro.sample("obs", dist.Normal(theta, 1.0))

tensor([ 59.7300,  20.8363, -16.3221,  38.9358,  10.9666,   1.6052,  -6.2763,
        -15.5909, -61.4017,  18.0785])

In [36]:
logging.basicConfig(format='%(message)s', level=logging.INFO)
pyro.enable_validation(__debug__)
pyro.set_rng_seed(0)


def model(sigma):
    eta = pyro.sample('eta', dist.Normal(torch.zeros(10), torch.ones(10)))
    mu = pyro.sample('mu', dist.Normal(torch.zeros(1), 10 * torch.ones(1)))
    tau = pyro.sample('tau', dist.HalfCauchy(scale=25 * torch.ones(1)))

    theta = mu + tau * eta

    return pyro.sample("obs", dist.Normal(theta, sigma))


def conditioned_model(model, sigma, y):
    return poutine.condition(model, data={"obs": y})(sigma)



nuts_kernel = NUTS(conditioned_model, jit_compile=False)
mcmc = MCMC(nuts_kernel,
            num_samples=1000,
            warmup_steps=1000,
            num_chains=4)
mcmc.run(model, 1.0, torch.ones(10))
mcmc.summary(prob=0.5)







                mean       std    median     25.0%     75.0%     n_eff     r_hat
    eta[0]      0.00      0.93     -0.02     -0.70      0.50   4186.91      1.00
    eta[1]     -0.00      0.95     -0.00     -0.58      0.69   4149.91      1.00
    eta[2]     -0.01      0.93     -0.00     -0.51      0.77   4440.24      1.00
    eta[3]      0.01      0.96     -0.02     -0.60      0.67   3823.44      1.00
    eta[4]      0.00      0.94     -0.01     -0.67      0.58   4654.07      1.00
    eta[5]      0.00      0.96      0.02     -0.62      0.65   4793.30      1.00
    eta[6]      0.01      0.98     -0.03     -0.65      0.65   4281.17      1.00
    eta[7]     -0.02      0.95     -0.01     -0.64      0.61   4875.79      1.00
    eta[8]     -0.00      0.95     -0.01     -0.62      0.66   4226.83      1.00
    eta[9]      0.00      0.98      0.00     -0.66      0.64   4891.93      1.00
     mu[0]      1.00      0.33      0.99      0.77      1.22   4272.22      1.00
    tau[0]      0.31   

In [44]:
mcmc.summary(prob=0.68)


                mean       std    median     16.0%     84.0%     n_eff     r_hat
    eta[0]      0.00      0.93     -0.02     -0.84      0.94   4186.91      1.00
    eta[1]     -0.00      0.95     -0.00     -1.05      0.81   4149.91      1.00
    eta[2]     -0.01      0.93     -0.00     -0.96      0.89   4440.24      1.00
    eta[3]      0.01      0.96     -0.02     -0.97      0.93   3823.44      1.00
    eta[4]      0.00      0.94     -0.01     -0.93      0.91   4654.07      1.00
    eta[5]      0.00      0.96      0.02     -0.83      1.04   4793.30      1.00
    eta[6]      0.01      0.98     -0.03     -0.96      0.96   4281.17      1.00
    eta[7]     -0.02      0.95     -0.01     -0.94      0.94   4875.79      1.00
    eta[8]     -0.00      0.95     -0.01     -1.02      0.87   4226.83      1.00
    eta[9]      0.00      0.98      0.00     -1.02      0.90   4891.93      1.00
     mu[0]      1.00      0.33      0.99      0.62      1.27   4272.22      1.00
    tau[0]      0.31      0

In [46]:
dist.Normal(0, 0.1)

Normal(loc: 0.0, scale: 0.10000000149011612)

In [43]:
?MCMC.summary

Signature: MCMC.summary(self, prob=0.9)
Docstring:
Prints a summary table displaying diagnostics of samples obtained from
posterior. The diagnostics displayed are mean, standard deviation, median,
the 90% Credibility Interval, :func:`~pyro.ops.stats.effective_sample_size`,
:func:`~pyro.ops.stats.split_gelman_rubin`.

:param float prob: the probability mass of samples within the credibility interval.
File:      ~/anaconda3/envs/syssynbio/lib/python3.7/site-packages/pyro/infer/mcmc/api.py
Type:      function


In [40]:
poutine.condition(model, data={"obs": torch.ones(10) * 2})(1.0)

tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

In [41]:
?NUTS

Init signature:
NUTS(
    model=None,
    potential_fn=None,
    step_size=1,
    adapt_step_size=True,
    adapt_mass_matrix=True,
    full_mass=False,
    use_multinomial_sampling=True,
    transforms=None,
    max_plate_nesting=None,
    jit_compile=False,
    jit_options=None,
    ignore_jit_warnings=False,
    target_accept_prob=0.8,
    max_tree_depth=10,
    init_strategy=<function init_to_uniform at 0x7fea683b2560>,
)
Docstring:     
No-U-Turn Sampler kernel, which provides an efficient and convenient way
to run Hamiltonian Monte Carlo. The number of steps taken by the
integrator is dynamically adjusted on each call to ``sample`` to ensure
an optimal length for the Hamiltonian trajectory [1]. As such, the samples
generated will typically have lower autocorrelation than those generated
by the :class:`~pyro.infer.mcmc.HMC` kernel. Optionally, the NUTS kernel
also provides the ability to adapt step size during the warmup phase.

Refer to the `baseball example <https://github.com/p

In [42]:
?MCMC

Init signature:
MCMC(
    kernel,
    num_samples,
    warmup_steps=None,
    initial_params=None,
    num_chains=1,
    hook_fn=None,
    mp_context=None,
    disable_progbar=False,
    disable_validation=True,
    transforms=None,
)
Docstring:     
Wrapper class for Markov Chain Monte Carlo algorithms. Specific MCMC algorithms
are TraceKernel instances and need to be supplied as a ``kernel`` argument
to the constructor.

.. note:: The case of `num_chains > 1` uses python multiprocessing to
    run parallel chains in multiple processes. This goes with the usual
    caveats around multiprocessing in python, e.g. the model used to
    initialize the ``kernel`` must be serializable via `pickle`, and the
    performance / constraints will be platform dependent (e.g. only
    the "spawn" context is available in Windows). This has also not
    been extensively tested on the Windows platform.

:param kernel: An instance of the ``TraceKernel`` class, which when
    given an execution trace re